In [1]:
from google.colab import files
uploaded = files.upload()


Saving hotel_new.csv to hotel_new.csv


In [2]:
import pandas
import pandas as pd
import numpy
from numpy import array
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.layers import Embedding
#from keras.preprocessing import sequence
from keras.models import load_model
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import nltk

In [3]:
reviews_df=pd.read_csv("hotel_new.csv")

In [4]:
reviews_df["review"] = reviews_df["Negative_Review"] + reviews_df["Positive_Review"]
reviews_df["is_bad_review"] = reviews_df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)

In [5]:
reviews_df = reviews_df[["review", "is_bad_review"]]
reviews_df.head(10)


,review,is_bad_review
0,I am so angry that i made this post available...,1
1,"No Negative No real complaints, the hotel was ...",0
2,Rooms are nice but for elderly a bit difficul...,0
3,My room was dirty and I was afraid to walk ba...,1
4,You When I booked with your company on line y...,0
5,Backyard of the hotel is total mess shouldn t...,0
6,Cleaner did not change our sheet and duvet ev...,1
7,Apart from the price for the brekfast Everyth...,0
8,Even though the pictures show very clean room...,0
9,The aircondition makes so much noise and its ...,0


In [6]:
reviews_df = reviews_df.sample(frac = 0.1, replace = False, random_state=42)


In [7]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
import string
from nltk.tokenize import  WhitespaceTokenizer
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

def clean_text(text):
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text(reducing words to its base form)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
nltk.download('stopwords')
nltk.download('wordnet')
reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))
reviews_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,review,is_bad_review,review_clean
3543,No Negative Great value for business overnight...,0,negative great value business overnight minute...
5731,Room was nice and contemporary the bathroom s...,0,room nice contemporary bathroom slide glass do...
4801,room was tired in parts and expensive for wha...,0,room tire part expensive get great location lo...
416,Lower ground floor room L01 possibly the smal...,0,low ground floor room possibly small bathroom ...
5125,Had to ask a member of staff to sort out the ...,0,ask member staff sort tv couldnt get work loca...


In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
#vader returns positivity, negativity, neutrality score
reviews_df["sentiments"] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))
reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [11]:
reviews_df["nb_chars"] = reviews_df["review"].apply(lambda x: len(x))

# add number of words column
reviews_df["nb_words"] = reviews_df["review"].apply(lambda x: len(x.split(" ")))
reviews_df.head(10)

,review,is_bad_review,review_clean,neg,neu,pos,compound,nb_chars,nb_words
3543,No Negative Great value for business overnight...,0,negative great value business overnight minute...,0.252,0.470,0.278,0.1531,94,16
5731,Room was nice and contemporary the bathroom s...,0,room nice contemporary bathroom slide glass do...,0.018,0.853,0.129,0.9465,839,161
4801,room was tired in parts and expensive for wha...,0,room tire part expensive get great location lo...,0.122,0.546,0.332,0.7184,84,17
416,Lower ground floor room L01 possibly the smal...,0,low ground floor room possibly small bathroom ...,0.171,0.659,0.170,0.2425,205,39
5125,Had to ask a member of staff to sort out the ...,0,ask member staff sort tv couldnt get work loca...,0.000,0.866,0.134,0.5267,113,26
9915,Room size was small spacious enough for a nig...,0,room size small spacious enough night two exce...,0.120,0.471,0.410,0.9095,134,24
293,Nothing it was perfect Very clean and tidy ho...,0,nothing perfect clean tidy hotel comfortable b...,0.121,0.446,0.433,0.8266,91,16
5020,Food in bar was lukewarm Breakfast mediocre S...,0,food bar lukewarm breakfast mediocre small roo...,0.000,1.000,0.000,0.0000,84,15
1765,not enough tea coffee 30 charge on room a day...,0,enough tea coffee charge room day scar touch u...,0.123,0.723,0.153,0.1779,99,21
4192,room key was older strip type which failed to...,0,room key old strip type fail work even replace...,0.127,0.808,0.066,-0.4215,223,43


In [12]:
reviews_df['Rating'] = reviews_df.apply(lambda row: 'Pos' if row['pos'] > row['neg'] else 'Neg', axis=1)


In [13]:
reviews_df.head(10)

,review,is_bad_review,review_clean,neg,neu,pos,compound,nb_chars,nb_words,Rating
3543,No Negative Great value for business overnight...,0,negative great value business overnight minute...,0.252,0.470,0.278,0.1531,94,16,Pos
5731,Room was nice and contemporary the bathroom s...,0,room nice contemporary bathroom slide glass do...,0.018,0.853,0.129,0.9465,839,161,Pos
4801,room was tired in parts and expensive for wha...,0,room tire part expensive get great location lo...,0.122,0.546,0.332,0.7184,84,17,Pos
416,Lower ground floor room L01 possibly the smal...,0,low ground floor room possibly small bathroom ...,0.171,0.659,0.170,0.2425,205,39,Neg
5125,Had to ask a member of staff to sort out the ...,0,ask member staff sort tv couldnt get work loca...,0.000,0.866,0.134,0.5267,113,26,Pos
9915,Room size was small spacious enough for a nig...,0,room size small spacious enough night two exce...,0.120,0.471,0.410,0.9095,134,24,Pos
293,Nothing it was perfect Very clean and tidy ho...,0,nothing perfect clean tidy hotel comfortable b...,0.121,0.446,0.433,0.8266,91,16,Pos
5020,Food in bar was lukewarm Breakfast mediocre S...,0,food bar lukewarm breakfast mediocre small roo...,0.000,1.000,0.000,0.0000,84,15,Neg
1765,not enough tea coffee 30 charge on room a day...,0,enough tea coffee charge room day scar touch u...,0.123,0.723,0.153,0.1779,99,21,Pos
4192,room key was older strip type which failed to...,0,room key old strip type fail work even replace...,0.127,0.808,0.066,-0.4215,223,43,Neg


In [32]:
x = reviews_df['review_clean']
y = reviews_df['Rating']

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=1,test_size=0.2)


In [15]:
nltk.download('punkt')
sent_len = []
for sent in reviews_df['review']:
    sent_len.append(len(word_tokenize(sent)))
reviews_df['sent_len'] = sent_len
reviews_df.head()
max_len=(max(sent_len))




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(xtrain)
tok.index_word

In [17]:
sequences_train = tok.texts_to_sequences(xtrain)

In [18]:
vocab_len = len(tok.index_word)

In [19]:
from tensorflow.keras.preprocessing import sequence
sequence_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)
sequence_matrix_train

array([[   0,    0,    0, ...,   30,  578,   16],
       [   0,    0,    0, ...,   66,  152,  259],
       [   0,    0,    0, ...,    1,   94,   20],
       ...,
       [   0,    0,    0, ..., 1090, 1252,   20],
       [   0,    0,    0, ..., 1195,   75,  357],
       [   0,    0,    0, ...,    8,  529,    1]], dtype=int32)

In [20]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ytrain = lb.fit_transform(ytrain)
ytest = lb.transform(ytest)

In [21]:
from keras import regularizers
import tensorflow as tf

model = Sequential()
# Embedding: to learn a dense representation of words or categorical data. Vocabulary length,output_dim,mask_zero to handle padding sequences
model.add(Embedding(vocab_len+1,500,input_length=max_len,mask_zero=True))
#tanh squashes the input values between -1 to 1, it is zero-centered
model.add(LSTM(16,activation='tanh'))                                                                                      # LSTM Layer
model.add(Dense(8,activation='relu',kernel_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001)))    # Hidden Layer
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

In [22]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
trained_model = model.fit(sequence_matrix_train,ytrain,epochs=50)

Epoch 1/50
26/26 [==============================] - 16s 374ms/step - loss: 1.1015 - accuracy: 0.5218
Epoch 2/50
26/26 [==============================] - 10s 376ms/step - loss: 1.0789 - accuracy: 0.6729
Epoch 3/50
26/26 [==============================] - 11s 413ms/step - loss: 1.0554 - accuracy: 0.6679
Epoch 4/50
26/26 [==============================] - 22s 858ms/step - loss: 1.0273 - accuracy: 0.6891
Epoch 5/50
26/26 [==============================] - 18s 660ms/step - loss: 0.9875 - accuracy: 0.6954
Epoch 6/50
26/26 [==============================] - 11s 411ms/step - loss: 0.9499 - accuracy: 0.6742
Epoch 7/50
26/26 [==============================] - 11s 414ms/step - loss: 0.9042 - accuracy: 0.6941
Epoch 8/50
26/26 [==============================] - 10s 393ms/step - loss: 0.8515 - accuracy: 0.7029
Epoch 9/50
26/26 [==============================] - 10s 372ms/step - loss: 0.8139 - accuracy: 0.6891
Epoch 10/50
26/26 [==============================] - 11s 418ms/step - loss: 0.7954 - accura

In [23]:
sequence_matrix_test = sequence.pad_sequences(tok.texts_to_sequences(xtest),maxlen=max_len)

print('Training_loss :',model.evaluate(sequence_matrix_train,ytrain))
print('Testing_loss :',model.evaluate(sequence_matrix_test,ytest))


26/26 [==============================] - 4s 110ms/step - loss: 0.1476 - accuracy: 0.9963
Training_loss : [0.1475500464439392, 0.9962546825408936]
7/7 [==============================] - 1s 79ms/step - loss: 0.4264 - accuracy: 0.8408
Testing_loss : [0.4263856112957001, 0.8407959938049316]


In [24]:
import numpy as np
Y_pred = model.predict(sequence_matrix_test)
print(np.round(Y_pred,3))

Y_pred = [np.argmax(i) for i in Y_pred]

7/7 [==============================] - 2s 76ms/step
[[0.074 0.914 0.012]
 [0.74  0.236 0.024]
 [0.186 0.79  0.025]
 [0.053 0.941 0.006]
 [0.048 0.946 0.006]
 [0.121 0.87  0.009]
 [0.546 0.42  0.034]
 [0.049 0.945 0.006]
 [0.072 0.916 0.012]
 [0.119 0.873 0.008]
 [0.189 0.793 0.019]
 [0.072 0.916 0.012]
 [0.063 0.928 0.01 ]
 [0.064 0.926 0.01 ]
 [0.047 0.947 0.006]
 [0.072 0.918 0.01 ]
 [0.041 0.955 0.005]
 [0.042 0.954 0.004]
 [0.044 0.951 0.005]
 [0.094 0.895 0.01 ]
 [0.197 0.79  0.013]
 [0.039 0.957 0.004]
 [0.656 0.234 0.11 ]
 [0.064 0.927 0.009]
 [0.101 0.887 0.012]
 [0.086 0.902 0.012]
 [0.719 0.201 0.08 ]
 [0.04  0.956 0.004]
 [0.502 0.484 0.015]
 [0.075 0.914 0.011]
 [0.512 0.362 0.125]
 [0.045 0.95  0.006]
 [0.053 0.94  0.007]
 [0.044 0.95  0.005]
 [0.295 0.692 0.014]
 [0.294 0.673 0.033]
 [0.05  0.943 0.007]
 [0.493 0.4   0.107]
 [0.054 0.939 0.008]
 [0.053 0.94  0.007]
 [0.131 0.862 0.008]
 [0.045 0.949 0.006]
 [0.056 0.937 0.007]
 [0.095 0.898 0.007]
 [0.178 0.792 0.03 ]
 [0

In [25]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(ytest,Y_pred))
print(confusion_matrix(ytest,Y_pred))

              precision    recall  f1-score   support

           0       0.76      0.47      0.58        47
           1       0.85      0.95      0.90       154

    accuracy                           0.84       201
   macro avg       0.81      0.71      0.74       201
weighted avg       0.83      0.84      0.83       201

[[ 22  25]
 [  7 147]]


In [48]:
y_true = reviews_df['Rating'].apply(lambda x: 1 if 'Pos' else 0)
#  y_true is a list of correct outputs and y_pred is a list of predicted outputs
correct_predictions = sum(1 for true, pred in zip(y_true, Y_pred) if true == pred)
print(f"Number of Correct Predictions: {correct_predictions}")
tot_pred=len(Y_pred)
accuracy= (correct_predictions/tot_pred)*100
print(accuracy)






Number of Correct Predictions: 172
85.57213930348259


In [26]:
def dl_predict(text):
    cleantext = clean_text(text)
    seq = tok.texts_to_sequences([cleantext])
    padded = sequence.pad_sequences(seq)

    pred = model.predict(padded)
    # Get the index of the maximum value in the prediction array
    predicted_index = np.argmax(pred, axis=1)[0]
    # Get the label name using the index
    result = lb.classes_[predicted_index]

    return result

In [27]:
text="I am in awe of how clean the rooms were and how friendly the staff was!"

print('Prediction using DNN: {}'.format(dl_predict(text)))

1/1 [==============================] - 2s 2s/step
Prediction using DNN: Pos


In [28]:
model.save("LSTM.h5")
!ls

hotel_new.csv  LSTM.h5	sample_data
